In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import re
from pathlib import Path
from scipy import stats
import sys, os
# --- bring in the styler
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))
from md_styler import MDStyler
sty = MDStyler().apply()
NMOLS = 200

## Discover data

In [3]:
def load_radius_box(radius_path: Path, box_path: Path):
    """Load a pair of radius/box-z CSVs and return a DataFrame."""
    r_df = pd.read_csv(radius_path)
    b_df = pd.read_csv(box_path)

    # Basic sanity: assume same time grid; align on 'Time (ps)'
    df = pd.merge(r_df, b_df, on="Time (ps)", how="inner")

    df.rename(columns={
        "Radius (Å)": "radius_A",
        "Z-dimension (Å)": "box_A",
    }, inplace=True)

    return df


def compute_observables(df: pd.DataFrame, n_motors: int = 200):
    """Add diameter [nm] and axial density [motors / nm]."""
    radius_nm = df["radius_A"].to_numpy() / 10
    box_nm = df["box_A"].to_numpy() / 10

    diameter_nm = 2.0 * radius_nm
    axial_density = n_motors / box_nm  # motors / nm

    return df["Time (ps)"].to_numpy(), diameter_nm, axial_density

In [21]:
def time_piecewise_to_unit(t_ps, t_split_ps=20.0, t_max_ps=100_000.0):
    """
    Map physical times (in ps) to a unit axis [0, 1] via a two-segment mapping:
        [0, t_split]      -> [0, 0.5]
        [t_split, t_max]  -> [0.5, 1]
    """
    t = np.asarray(t_ps, float)
    x = np.empty_like(t)

    mask_left = t <= t_split_ps
    mask_right = ~mask_left

    x[mask_left] = 0.5 * (t[mask_left] / t_split_ps)
    x[mask_right] = 0.5 + 0.5 * ((t[mask_right] - t_split_ps) / (t_max_ps - t_split_ps))

    return x


def physical_times_for_trigger_and_cont(t_trigger_ps,
                                        t_cont_raw_ps,
                                        t_split_ps=20.0,
                                        t_second_end_ns=100.0):
    """
    Construct 'physical' times (in ps) for trigger and continuation.

    Trigger:      assumed to span 0–20 ps already.
    Continuation: rescaled so that its range maps to [20 ps, t_second_end_ns].
    """
    t2_end_ps = t_second_end_ns * 1000  # 100 ns -> 100000 ps

    t_trig_phys = np.asarray(t_trigger_ps, float)

    t_cont_raw = np.asarray(t_cont_raw_ps, float)
    if t_cont_raw.max() == 0:
        raise ValueError("Continuation timeseries has max time 0 ps.")

    # linearly stretch raw continuation times to [t_split_ps, t2_end_ps]
    t_cont_phys = t_split_ps + (t_cont_raw / t_cont_raw.max()) * (t2_end_ps - t_split_ps)

    return t_trig_phys, t_cont_phys, t2_end_ps


def set_two_segment_xticks(
    ax,
    t_split_ps: float = 20.0,
    t_second_end_ns: float = 100.0,
):
    """
    Two-piece x-axis ticks:
      left:  0, 10, 20 ps
      right: 25, 50, 75, 100 ns
    """
    # total max time in ps (e.g. 100 ns -> 100000 ps)
    t_max_ps = t_second_end_ns * 1000

    left_ticks_ps = np.array([0.0, 10.0, 20.0])
    right_ticks_ns = np.array([25.0, 50.0, 75.0, 100.0])

    tick_times_ps = list(left_ticks_ps) + list(right_ticks_ns * 1000)

    tick_positions = time_piecewise_to_unit(
        tick_times_ps,
        t_split_ps=t_split_ps,
        t_max_ps=t_max_ps,
    )

    labels = []
    for t_ps in tick_times_ps:
        if t_ps <= t_split_ps + 1e-9:
            labels.append(f"{t_ps:.0f} ps")
        else:
            ns = t_ps / 1000
            labels.append(f"{ns:.0f} ns")

    ax.set_xticks(tick_positions)
    ax.set_xticklabels(labels)



In [28]:
def plot_triggers_and_continuation(
    trigger_glob: str,
    cont_radius_path: Path = Path("../data/cont_radius.csv"),
    cont_box_path: Path = Path("../data/cont_box-z.csv"),
    t_split_ps: float = 20.0,
    t_second_end_ns: float = 100.0,
    n_motors: int = 200,
):
    # --- gather files ---
    trigger_radius_files = sorted(Path("../data").glob(trigger_glob))
    if len(trigger_radius_files) == 0:
        raise FileNotFoundError(f"No trigger radius files found for glob: {trigger_glob}")

    # load continuation
    cont_df = load_radius_box(cont_radius_path, cont_box_path)
    t_cont_raw_ps, diam_cont_nm, dens_cont = compute_observables(cont_df, n_motors=n_motors)

    # use first trigger's time grid for tick scaling etc.
    first_trigger_df = load_radius_box(
        trigger_radius_files[0],
        Path("../data/350_box-z.csv")
    )
    t_trig_ps_first, _, _ = compute_observables(first_trigger_df, n_motors=n_motors)

    t_trig_phys_first, t_cont_phys, t_max_ps = physical_times_for_trigger_and_cont(
        t_trig_ps_first, t_cont_raw_ps,
        t_split_ps=t_split_ps,
        t_second_end_ns=t_second_end_ns
    )

    # x positions for continuation
    x_cont = time_piecewise_to_unit(
        t_cont_phys,
        t_split_ps=t_split_ps,
        t_max_ps=t_max_ps
    )

    # --- figure & axes (horizontal, 16:9 content) ---
    fig, ax_dens = sty.fig_horizontal()
    ax_diam = ax_dens.twinx()

    # Colors from styler
    c_dens = sty.get_color("cyan")
    c_diam = sty.get_color("orange")
    c_split = sty.get_color("black")

    # use styler's rc default line width
    default_lw = plt.rcParams["lines.linewidth"]
    lw_trig = default_lw                # triggers
    lw_cont = default_lw * 1.2          # continuation slightly emphasized

    # --- plot triggers (all) ---
    for radius_path in trigger_radius_files:
        box_path = Path("../data/350_box-z.csv")
        if not box_path.exists():
            print(f"Warning: no box-z file for {radius_path}, skipping.")
            continue

        df_trig = load_radius_box(radius_path, box_path)
        t_trig_ps, diam_trig_nm, dens_trig = compute_observables(df_trig, n_motors=n_motors)

        # Build physical times and map to [0,1]
        t_trig_phys, _, _ = physical_times_for_trigger_and_cont(
            t_trig_ps, t_cont_raw_ps,
            t_split_ps=t_split_ps,
            t_second_end_ns=t_second_end_ns
        )
        x_trig = time_piecewise_to_unit(
            t_trig_phys,
            t_split_ps=t_split_ps,
            t_max_ps=t_max_ps
        )

        ax_dens.plot(
            x_trig, dens_trig,
            color=c_dens,
            linewidth=lw_trig,
            label="_nolabel_"
        )
        ax_diam.plot(
            x_trig, diam_trig_nm,
            color=c_diam,
            linewidth=lw_trig,
            label="_nolabel_"
        )

    # --- plot continuation (highlighted) ---
    ax_dens.plot(
        x_cont, dens_cont,
        color=c_dens,
        linewidth=lw_cont,
        label="Axial density"
    )
    ax_diam.plot(
        x_cont, diam_cont_nm,
        color=c_diam,
        linewidth=lw_cont,
        label="Diameter"
    )

    # --- split marker at logical transition (x = 0.5) ---
    ax_dens.axvline(0.5, color=c_split, linestyle="--", linewidth=default_lw * 0.7)
    ax_dens.text(
        0.5, 1.02, "x-scale change",
        transform=ax_dens.transAxes,
        ha="center", va="bottom",
        fontsize=sty.base_fontsize * 0.85
    )

    # --- axes labels ---
    ax_dens.set_xlabel("Time")
    ax_dens.set_ylabel("Axial density (motors / nm)", color=c_dens)
    ax_diam.set_ylabel("Diameter (nm)", color=c_diam)

    ax_dens.tick_params(axis="y", colors=c_dens)
    ax_diam.tick_params(axis="y", colors=c_diam)

    # --- custom x ticks (0,10,20 ps; 25,50,75,100 ns) ---
    set_two_segment_xticks(
        ax_dens,
        t_split_ps=t_split_ps,
        t_second_end_ns=t_second_end_ns,
    )

    # --- legend (one entry per observable) ---
    from matplotlib.lines import Line2D
    handles = [
        Line2D([], [], color=c_dens, lw=lw_cont, label="Axial density"),
        Line2D([], [], color=c_diam, lw=lw_cont, label="Diameter"),
    ]
    ax_dens.legend(handles=handles, frameon=False)

    fig.tight_layout()
    return fig, (ax_dens, ax_diam)


In [ ]:
fig, (ax_dens, ax_diam) = plot_triggers_and_continuation(
    trigger_glob="150_2_radius.csv",   # or e.g. "[0-9]*_radius.csv"
    cont_radius_path=Path("../data/cont_2_radius.csv"),
    cont_box_path=Path("../data/cont_box-z.csv"),
    t_split_ps=20.0,
    t_second_end_ns=100.0,
    n_motors=200,
)


In [34]:
def plot_triggers_and_continuation_logx(
    trigger_glob: str,
    cont_radius_path: Path = Path("../data/cont_radius.csv"),
    cont_box_path: Path = Path("../data/cont_box-z.csv"),
    t_split_ps: float = 20.0,
    t_second_end_ns: float = 100.0,
    n_motors: int = 200,
):
    """
    Concatenate triggers + continuation, then plot on a log-scaled x-axis.
    No piecewise compression — pure physical time in ps.
    """

    # --- gather trigger files ---
    trigger_radius_files = sorted(Path("../data").glob(trigger_glob))
    if len(trigger_radius_files) == 0:
        raise FileNotFoundError(f"No trigger files match: {trigger_glob}")

    # --- load continuation ---
    cont_df = load_radius_box(cont_radius_path, cont_box_path)
    t_cont_raw_ps, diam_cont_nm, dens_cont = compute_observables(cont_df, n_motors=n_motors)

    # Stretch continuation: map its 0–max raw time → 20 ps to 100 ns
    t_max_ps = t_second_end_ns * 1000  # 100 ns → 100000 ps

    t_cont_phys_ps = t_split_ps + (t_cont_raw_ps / t_cont_raw_ps.max()) * (t_max_ps - t_split_ps)

    # --- prepare figure ---
    fig, ax_dens = sty.fig_horizontal()
    ax_diam = ax_dens.twinx()

    # colors
    c_dens = sty.get_color("cyan")
    c_diam = sty.get_color("orange")

    default_lw = plt.rcParams["lines.linewidth"]
    lw_trig = default_lw
    lw_cont = default_lw * 1.2

    # --- plot triggers first ---
    for radius_path in trigger_radius_files:
        box_path = Path("../data/350_box-z.csv")

        df_trig = load_radius_box(radius_path, box_path)
        t_trig_ps, diam_trig_nm, dens_trig = compute_observables(df_trig, n_motors=n_motors)

        # Plot trigger data in physical ps
        ax_dens.plot(
            t_trig_ps, dens_trig,
            color=c_dens,
            linewidth=lw_trig,
            alpha=0.8,
            label="_nolabel_"
        )

        ax_diam.plot(
            t_trig_ps, diam_trig_nm,
            color=c_diam,
            linewidth=lw_trig,
            alpha=0.8,
            label="_nolabel_"
        )

    # --- plot continuation on physical log-time ---
    ax_dens.plot(
        t_cont_phys_ps, dens_cont,
        color=c_dens,
        linewidth=lw_cont,
        label="Axial density"
    )

    ax_diam.plot(
        t_cont_phys_ps, diam_cont_nm,
        color=c_diam,
        linewidth=lw_cont,
        label="Diameter"
    )

    # --- x-axis log scale ---
    ax_dens.set_xscale("log")
    ax_dens.set_xlim([1,1e5])

    # Label axes
    ax_dens.set_xlabel("Time (ps, log scale)")
    ax_dens.set_ylabel("Axial density (motors / nm)", color=c_dens)
    ax_diam.set_ylabel("Diameter (nm)", color=c_diam)

    ax_dens.tick_params(axis="y", colors=c_dens)
    ax_diam.tick_params(axis="y", colors=c_diam)

    

    # --- nice log ticks chosen automatically ---
    # If you want explicit ticks, uncomment and tune:
    # ax_dens.set_xticks([1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5])
    # ax_dens.get_xaxis().set_major_formatter(mpl.ticker.ScalarFormatter())

    # --- legend ---
    from matplotlib.lines import Line2D
    handles = [
        Line2D([], [], color=c_dens, lw=lw_cont, label="Axial density"),
        Line2D([], [], color=c_diam, lw=lw_cont, label="Diameter"),
    ]
    ax_dens.legend(handles=handles, frameon=False, loc="upper right")

    fig.tight_layout()
    return fig, (ax_dens, ax_diam)

In [ ]:
fig, _ = plot_triggers_and_continuation_logx("150_radius.csv")


In [43]:
def compute_change_from_start_to_continuation(
    trigger_radius_path=Path("../data/150_2_radius.csv"),
    trigger_box_path=Path("../data/350_box-z.csv"),
    cont_radius_path=Path("../data/cont_2_radius.csv"),
    cont_box_path=Path("../data/cont_box-z.csv"),
    n_motors: int = 200,
):
    """
    Computes net & percent change in:
        - diameter (nm)
        - axial density (motors/nm)

    Between:
        (A) trigger at t = 0 ps
        (B) continuation averaged over last 10% of data
    """

    # --- load trigger at t = 0 ---
    trig_df = load_radius_box(trigger_radius_path, trigger_box_path)
    t_trig_ps, diam_trig_nm, dens_trig = compute_observables(trig_df, n_motors=n_motors)

    # t=0 is first entry
    diam_0 = diam_trig_nm[0]
    dens_0 = dens_trig[0]

    # --- load continuation ---
    cont_df = load_radius_box(cont_radius_path, cont_box_path)
    t_cont_ps, diam_cont_nm, dens_cont = compute_observables(cont_df, n_motors=n_motors)

    # average over last 50%
    n = len(diam_cont_nm)
    k0 = int(0.5 * n)

    diam_end = np.mean(diam_cont_nm[k0:])
    dens_end = np.mean(dens_cont[k0:])

    # --- compute changes ---
    diam_net = diam_end - diam_0
    dens_net = dens_end - dens_0

    diam_pct = 100.0 * diam_net / diam_0
    dens_pct = 100.0 * dens_net / dens_0

    # --- return dictionary for easy printing/logging ---
    return {
        "diameter_start_nm": diam_0,
        "diameter_end_nm": diam_end,
        "diameter_net_nm": diam_net,
        "diameter_percent_change": diam_pct,

        "density_start": dens_0,
        "density_end": dens_end,
        "density_net": dens_net,
        "density_percent_change": dens_pct,
    }


In [44]:
results = compute_change_from_start_to_continuation()
results


{'diameter_start_nm': np.float64(3.9997949927943637),
 'diameter_end_nm': np.float64(4.110628372657623),
 'diameter_net_nm': np.float64(0.11083337986325947),
 'diameter_percent_change': np.float64(2.7709765141194977),
 'density_start': np.float64(17.302373158897726),
 'density_end': np.float64(18.421800395981986),
 'density_net': np.float64(1.1194272370842597),
 'density_percent_change': np.float64(6.4697901657992825)}